In [1]:
import requests
from requests import get
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# define the url that you are scraping
url = 'https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prv'

In [3]:
# query the website and return the html to the variable 'page'
page = urllib.request.urlopen(url)

# parse the html using beautiful soup and store in variable 'soup'
soup = BeautifulSoup(page, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   IMDb "Top 1000"
(Sorted by Popularity Ascending) - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "Loa

#### Here is the information we’ll gather from each movie listing:
- The title
- The year it was released
- How long the movie is
- IMDb’s rating of the movie
- The Metascore of the movie
- How many votes the movie got
- The U.S. gross earnings of the movie

In [4]:
#initialize empty lists where you'll store your data
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [5]:
movie_div = soup.find_all('div', class_='lister-item mode-advanced')

In [6]:
#initiate the for loop 
#this tells your scraper to iterate through 
#every div container we stored in move_div
for container in movie_div:
    # name
    name = container.h3.a.text
    titles.append(name)
    
    # year
    year = container.h3.find('span', class_='lister-item-year').text
    years.append(year)
    
    # runtime
    runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else ''
    time.append(runtime)
    
    # rating
    imdb = float(container.strong.text)
    imdb_ratings.append(imdb)
    
    # metascore
    m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else ''
    metascores.append(m_score)
    
    
    nv = container.find_all('span', attrs={'name': 'nv'})
    
    # filter nv for votes 
    vote = nv[0].text
    votes.append(vote)
     
    # filter nv for gross    
    grosses = nv[1].text if len(nv) > 1 else ''
    us_gross.append(grosses)
    
    print(titles)
    print(years)
    print(time)
    print(imdb_ratings)
    print(metascores)
    print(votes)
    print(us_gross)

['Dil Bechara']
['(2020)']
['101 min']
[9.1]
['']
['93,836']
['']
['Dil Bechara', 'Hamilton']
['(2020)', '(2020)']
['101 min', '160 min']
[9.1, 8.8]
['', '90        ']
['93,836', '31,947']
['', '']
['Dil Bechara', 'Hamilton', 'Knives Out']
['(2020)', '(2020)', '(2019)']
['101 min', '160 min', '130 min']
[9.1, 8.8, 7.9]
['', '90        ', '82        ']
['93,836', '31,947', '360,942']
['', '', '$165.36M']
['Dil Bechara', 'Hamilton', 'Knives Out', 'Gone with the Wind']
['(2020)', '(2020)', '(2019)', '(1939)']
['101 min', '160 min', '130 min', '238 min']
[9.1, 8.8, 7.9, 8.1]
['', '90        ', '82        ', '97        ']
['93,836', '31,947', '360,942', '282,828']
['', '', '$165.36M', '$198.68M']
['Dil Bechara', 'Hamilton', 'Knives Out', 'Gone with the Wind', 'The Gentlemen']
['(2020)', '(2020)', '(2019)', '(1939)', '(2019)']
['101 min', '160 min', '130 min', '238 min', '113 min']
[9.1, 8.8, 7.9, 8.1, 7.9]
['', '90        ', '82        ', '97        ', '51        ']
['93,836', '31,947', '36

In [7]:
movies = pd.DataFrame({
'Movie': titles,
'Year': years,
'Time(min)': time,
'imdb': imdb_ratings,
'Metascore': metascores,
'Votes': votes,
'USD(grossMillions)': us_gross,
})

In [8]:
movies.set_index('movie', inplace=True)

KeyError: "None of ['movie'] are in the columns"

In [ ]:
movies.head()

In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Movie               50 non-null     object 
 1   Year                50 non-null     object 
 2   Time(min)           50 non-null     object 
 3   imdb                50 non-null     float64
 4   Metascore           50 non-null     object 
 5   Votes               50 non-null     object 
 6   USD(grossMillions)  50 non-null     object 
dtypes: float64(1), object(6)
memory usage: 2.9+ KB


#### Data Cleaning

In [12]:
# (‘(\d+’) says to extract all the digits in the string
# The .astype(int) method converts the result to an integer
movies['Year'] = movies['Year'].astype(str).str.extract('(\d+)').astype(int)
movies['Time(min)'] = movies['Time(min)'].astype(str).str.extract('(\d+)').astype(int)
movies['Metascore'] = pd.to_numeric(movies['Metascore'], errors='coerce')
movies['Votes'] = movies['Votes'].astype(str).str.replace(',', '').astype(int)

# lstrip(‘$’).rstrip(‘M’) is our function arguments. This tells our function to strip the $ from the left side and strip the M from the right side.
movies['USD(grossMillions)'] = movies['USD(grossMillions)'].map(lambda x: x.lstrip('$').rstrip('M'))

# errors=’coerce’ will transform the nonnumeric values, our dashes, into NaN (not-a-number )values because we have dashes in place of the data that’s missing
movies['USD(grossMillions)'] = pd.to_numeric(movies['USD(grossMillions)'], errors='coerce')

In [13]:
movies

,Movie,Year,Time(min),imdb,Metascore,Votes,USD(grossMillions)
0,Dil Bechara,2020,101,9.1,NaN,93836,NaN
1,Hamilton,2020,160,8.8,90.0,31947,NaN
2,Knives Out,2019,130,7.9,82.0,360942,165.36
3,Gone with the Wind,1939,238,8.1,97.0,282828,198.68
4,The Gentlemen,2019,113,7.9,51.0,170416,NaN
5,The Shawshank Redemption,1994,142,9.3,80.0,2270396,28.34
6,Once Upon a Time... in Hollywood,2019,161,7.7,83.0,498631,142.50
7,The Godfather,1972,175,9.2,100.0,1566065,134.97
8,Joker,2019,122,8.5,59.0,838132,335.45
9,Gisaengchung,2019,132,8.6,96.0,458028,53.37
